In [4]:
data_path = "../data/info/SraRunInfo.csv"
SraAccList = "../data/info/SraAccList.csv"

In [5]:
import pandas as pd
df = pd.read_csv(data_path)
for index,item in df.iterrows():
    cmd_str = "wget -O {0}.sra {1}".format(item.Run,item.download_path)
    print(cmd_str)

wget -O SRR1916152.sra https://sra-downloadb.st-va.ncbi.nlm.nih.gov/sos2/sra-pub-run-6/SRR1916152/SRR1916152.1
wget -O SRR1916153.sra https://sra-downloadb.st-va.ncbi.nlm.nih.gov/sos2/sra-pub-run-6/SRR1916153/SRR1916153.1
wget -O SRR1916154.sra https://sra-downloadb.st-va.ncbi.nlm.nih.gov/sos2/sra-pub-run-6/SRR1916154/SRR1916154.1
wget -O SRR1916155.sra https://sra-downloadb.st-va.ncbi.nlm.nih.gov/sos2/sra-pub-run-6/SRR1916155/SRR1916155.1
wget -O SRR1916156.sra https://sra-downloadb.st-va.ncbi.nlm.nih.gov/sos2/sra-pub-run-6/SRR1916156/SRR1916156.1


In [9]:
for index,row in df.iterrows():
    cmd_str = "fastq-dump {0}.sra \nfastqc {0}.fastq".format(row[0])
    print(cmd_str)

fastq-dump SRR1916152.sra 
fastqc SRR1916152.fastq
fastq-dump SRR1916153.sra 
fastqc SRR1916153.fastq
fastq-dump SRR1916154.sra 
fastqc SRR1916154.fastq
fastq-dump SRR1916155.sra 
fastqc SRR1916155.fastq
fastq-dump SRR1916156.sra 
fastqc SRR1916156.fastq


### hisat2将reads比对到参考基因组

In [20]:
with open("../data/info/SraAccList.csv") as acc_file_handle:
    for line in acc_file_handle:
        line = line.strip()                                                       
        line = line.split("\t")
        cmd_str="hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/{0}.fastq | samtools view -bS - | samtools sort - -o {1}.bam".format(line[0],line[1])
        #cmd_str+="\nsamtools index {1}.bam
        print(cmd_str)

hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916152.fastq | samtools view -bS - | samtools sort - -o EV_3.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916153.fastq | samtools view -bS - | samtools sort - -o EV_4.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916154.fastq | samtools view -bS - | samtools sort - -o EV_2.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916155.fastq | samtools view -bS - | samtools sort - -o DNMT3B_3.bam
hisat2 -x ../../data/reference/yeast_ref -U ../../data/RAN_seq/SRR1916156.fastq | samtools view -bS - | samtools sort - -o DNMT3B_4.bam


### read count


In [3]:
with open(SraAccList) as acc_file_handle:
    for line in acc_file_handle:
        line = line.strip()                                                       
        line = line.split("\t")
        cmd_str="htseq-count -f bam -r pos ../read_aln/{1}.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > {1}.count.tab".format(line[0],line[1])
        #cmd_str+="\nsamtools index {1}.bam
        print(cmd_str)

htseq-count -f bam -r pos ../read_aln/EV_3.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > EV_3.count.tab
htseq-count -f bam -r pos ../read_aln/EV_4.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > EV_4.count.tab
htseq-count -f bam -r pos ../read_aln/DNMT3B_2.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > DNMT3B_2.count.tab
htseq-count -f bam -r pos ../read_aln/DNMT3B_3.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > DNMT3B_3.count.tab
htseq-count -f bam -r pos ../read_aln/DNMT3B_4.bam ../../data/reference/Saccharomyces_cerevisiae.R64-1-1.48.gtf > DNMT3B_4.count.tab


In [ ]:
## 讲reads count变换到一张表里

In [37]:
import os
base_dir = "../analysis/htseq-count"
dir_list = os.listdir(base_dir)
file_name_list = []
for i in dir_list:
    if os.path.splitext(i)[1] == '.tab':
        file_name_list.append(i)

# print(file_name_list)
count_dict = {}
for file_name in file_name_list:
    with open(base_dir+"/"+file_name) as f:
        for line in f:
            if line.startswith("__"):
                continue
            line = line.rstrip("\n")
            ele = line.split()
            if ele[0] in count_dict.keys():
                count_dict.get(ele[0]).append(ele[1])
            else:
                count_dict[ele[0]]=[ele[1]]
# print(count_dict)
result_str=""
#print("gene_id\t" + "\t".join([x[:-10] for x in file_name_list]))
result_str +="gene_id\t" + "\t".join([x[:-10] for x in file_name_list])+"\n"
for gene_id in count_dict:
#     print(gene_id+"\t"+"\t".join(count_dict[gene_id]))
    result_str+= gene_id+"\t"+"\t".join(count_dict[gene_id])+"\n"

print(result_str[:200])
with open("../data/info/result.csv","w") as f:
    f.write(result_str)

gene_id	EV_4	DNMT3B_4	DNMT3B_2	EV_3	DNMT3B_3
ETS1-1	24	18	32	51	17
ETS1-2	0	0	0	0	0
ETS2-1	0	0	0	0	0
ETS2-2	0	0	0	0	0
HRA1	30	14	19	71	24
ICR1	154	177	198	133	203
IRT1	260	491	618	387	555
ITS1-1	14	5	7	15	4
ITS1-2	0	0	0	0	0
ITS2-1	170	140	136	214	99
ITS2-2	0	0	0	0	0
LSR1	399	337	684	371	562
NME1	115	89	97	105	117
PWR1	25	34	40	28	41
Q0010	0	0	0	0	0
Q0017	0	0	0	0	0
Q0020	12	61	22	27	35
Q0032	0	0	0	0	0
Q0045	38	38	71	45	63
Q0050	46	50	77	50	74
Q0055	43	37	56	38	57
Q0060	0	1	1	1	1
Q0065	1	0	1	1	1
Q0070	0	0	0	0	0
Q0075	1	0	2	1	1
Q0080	0	0	0	0	0
Q0085	2	6	7	2	7
Q0092	0	0	0	0	0
Q0105	7	2	14	9	9
Q0110	0	0	0	0	0
Q0115	0	0	0	0	0
Q0120	1	0	1	0	3
Q0130	37	36	83	43	55
Q0140	0	0	0	0	0
Q0142	0	0	0	0	0
Q0143	0	0	0	0	0
Q0144	0	0	0	0	0
Q0158	33	144	52	46	82
Q0160	0	0	0	0	0
Q0182	0	0	0	0	0
Q0250	59	32	93	47	68
Q0255	2	1	2	0	2
Q0275	40	26	68	38	69
Q0285	0	0	0	0	0
Q0297	0	0	0	0	0
RDN18-1	0	0	0	0	0
RDN18-2	0	0	0	0	0
RDN25-1	0	0	0	0	0
RDN25-2	0	0	0	0	0
RDN37-1	0	0	0	0	0
RDN37-2	0	0	0	0	0
RDN5-1	0	0	0	0	0
RD